# Vizualization of the results

In this notebook we just visualize and format the pickle file containing experiments results

In [7]:
import pickle as pkl
import pandas as pd

pd.set_option("display.max_rows", 999)

In [14]:
def print_results(filename, metrics=None):
    with open(filename, "rb") as f:
        results = pkl.load(f)

    print("commit number %s: datetime: %s" % (results["commit"], results["datetime"]))

    if metrics is None:
        metrics = ["avg_prec_w", "avg_prec_w_train"]
    
    df = results["results"]

    return (
        df[["dataset", "classifier_title", *metrics, "repeat"]]
        .groupby(["dataset", "classifier_title", "repeat"])
        # .agg(["mean", "std"])
        .agg(["mean"])
        .reset_index()
        .loc[:, ["dataset", "classifier_title", *metrics]]    
        .pivot(index="dataset", columns="classifier_title")
        .style
        .format("{:.2f}")
        .background_gradient(axis="index")    
    )

In [24]:
filename = "../benchmarks_2021-04-19-23:24:51.pickle"
print_results(filename)

commit number fdbb940: datetime: 2021-04-19-23:24:51


## Resultats n_estimators=100, sans aggregation et max_features=None, dirichlet=0.0

In [17]:
filename = "../benchmarks_2021-04-19-15:25:42.pickle"

print_results(filename)

commit number 6ad6dfc: datetime: 2021-04-19-15:25:42


## Resultats n_estimators=100, sans aggregation et max_features="auto", dirichlet=0.0

In [20]:
filename = "../benchmarks_2021-04-19-15:29:20.pickle"
print_results(filename, metrics=["avg_prec_w", "fit_time"])

commit number 6ad6dfc: datetime: 2021-04-19-15:29:20


## Resultats n_estimators=100, sans aggregation et max_features="auto", dirichlet=0.5

In [21]:
filename = "../benchmarks_2021-04-19-15:33:27.pickle"
print_results(filename, metrics=["avg_prec_w", "fit_time"])

commit number 6ad6dfc: datetime: 2021-04-19-15:33:27


## Resultats n_estimators=100, avec aggregation et max_features="auto", dirichlet=0.5

In [22]:
filename = "../benchmarks_2021-04-19-15:37:09.pickle"
print_results(filename, metrics=["avg_prec_w", "fit_time"])

commit number 6ad6dfc: datetime: 2021-04-19-15:37:09


In [ ]:
import sys
import subprocess
from time import time
from datetime import datetime
import logging
import pickle as pkl
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    log_loss,
    accuracy_score,
)
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier

sys.path.extend([".", ".."])

from wildwood.dataset import loaders_small_classification, load_churn
from wildwood.forest import ForestClassifier


from wildwood.dataset import (
    load_adult,
    load_bank,
    load_breastcancer,
    load_car,
    load_cardio,
    load_churn,
    load_default_cb,
    load_letter,
    load_satimage,
    load_sensorless,
    load_spambase,
)

In [ ]:
dataset = load_bank()

In [ ]:
clf = ForestClassifier(
    n_estimators=1,
    n_jobs=1,
    class_weight="balanced",
    random_state=42,
    aggregation=False,
    max_features=None,
    dirichlet=0.0
)

In [ ]:
dataset.one_hot_encode = True
dataset.standardize = False
dataset.drop = None
X_train, X_test, y_train, y_test = dataset.extract(random_state=42)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
from bokeh.plotting import show, output_notebook
from wildwood.plot import plot_tree

output_notebook()

In [ ]:
fig = plot_tree(clf, height=900, width=900)

In [ ]:
show(fig)

In [ ]:
dataset.one_hot_encode = False
dataset.standardize = False
dataset.drop = None
X_train, X_test, y_train, y_test = dataset.extract(random_state=42)

In [ ]:
clf = ForestClassifier(
    n_estimators=1,
    n_jobs=1,
    class_weight="balanced",
    random_state=42,
    aggregation=False,
    max_features=None,
    dirichlet=0.0,
    categorical_features=dataset.categorical_features_
)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
fig = plot_tree(clf, height=900, width=900)

In [ ]:
show(fig)

In [ ]:
df = clf.get_nodes(0)

In [ ]:
pd.set_option("display.max_columns", 99)

In [ ]:
df.loc[338:342]

In [ ]:
clf.trees[0]._tree.

In [ ]:
partition_train = clf.trees[0]._tree_context.partition_train[19512:19517]

In [ ]:
partition_valid = clf.trees[0]._tree_context.partition_valid[11271:11273]

In [ ]:
pd.DataFrame(X_train[partition_train, :])

In [ ]:
pd.DataFrame(X_train[partition_valid, :])

In [ ]:
clf.trees[0]._tree_context.partition_train